Let's first import our most recent checkpoint - a clustered set of 600,000 images.

We'll then cluster another 100,000 using the same API and the same settings to get the "real" group to which this image should belong

Then we'll implement a FAISS database of the first 600,000 and then use this to predict the destination of the additional 100,000 images:

Comparing the two we see that the "correct" group was predicted x% of the time
Getting the groups for all 700,000 took x seconds, while predicting the additional 100,000 groups took y seconds. Of course as the problem scales we may have issues with VRAM or similar, but for now x...